In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import preproc

In [2]:
train = pd.read_csv('price_train.tsv', sep='\t')
house = pd.read_csv('price_housebase.tsv', sep='\t')

In [3]:
train.head()

,apartment_ratio,offers_in_sites_ratio,unique_active_buildings_count,avg_price_sqm,month,unique_active_building_series_count,rooms_offered_3_ratio,city_quadkey,median_price_sqm,rooms_offered_4_more_ratio,rooms_offered_1_ratio,total_area_avg,studio_ratio,offer_count,rooms_offered_2_ratio
0,0.0,0.0,1,22727.273438,2018-02-01,1,0.0,120132230303223,22727.273438,0.0,0.0,44.0,0.0,1,1.0
1,0.0,0.0,1,22727.273438,2018-03-01,1,0.0,120132230303223,22727.273438,0.0,0.0,44.0,0.0,1,1.0
2,0.0,0.0,1,43478.261719,2018-06-01,1,0.0,120132230332230,43478.261719,0.0,1.0,34.5,0.0,1,0.0
3,0.0,0.0,1,43478.261719,2018-07-01,1,0.0,120132230332230,43478.261719,0.0,1.0,34.5,0.0,1,0.0
4,0.0,0.0,1,36250.000000,2017-01-01,1,1.0,120132231322113,35714.285156,0.0,0.0,56.0,0.0,3,0.0


In [4]:
house.head()

,beauty_cnt,building_id,building_series_id,shopping_cnt,flats_count,latitude,building_type,cafe_restaurant_eating_out_cnt,unified_address,entertainment_cnt,...,art_cnt,site_id,healthcare_cnt,expect_demolition,longitude,ceiling_height,has_elevator,laundry_and_repair_services_cnt,build_year,city_quadkey
0,36.0,3031030817531317628,0,45.0,0,55.705219,BRICK,24.0,"Россия, Москва, 1-й Автозаводский проезд, 3",18.0,...,7.0,0,16.0,False,37.656994,0.00,True,38.0,1937,120310101302011
1,36.0,3031030817531317630,0,45.0,0,55.704845,BRICK,24.0,"Россия, Москва, 1-й Автозаводский проезд, 5",18.0,...,7.0,0,16.0,False,37.657295,0.00,False,38.0,1962,120310101302011
2,36.0,1764947998512909470,0,45.0,0,55.704697,UNKNOWN,24.0,"Россия, Москва, 1-й Автозаводский проезд, 7/1",18.0,...,7.0,0,16.0,False,37.658203,0.00,False,38.0,0,120310101302011
3,6.0,2614890160181791580,1564812,23.0,176,55.811642,BRICK,11.0,"Россия, Москва, 1-й Амбулаторный проезд, 2",16.0,...,4.0,0,14.0,False,37.535599,2.65,True,25.0,1967,120310101031222
4,6.0,4152249910202725078,1564812,23.0,60,55.810810,BRICK,11.0,"Россия, Москва, 1-й Амбулаторный проезд, 5к1",16.0,...,4.0,0,14.0,False,37.534649,0.00,False,25.0,1959,120310101031222


In [5]:
house['latlon'] = list(zip(house.latitude.values, house.longitude.values))

In [6]:
house.head(1)

,beauty_cnt,building_id,building_series_id,shopping_cnt,flats_count,latitude,building_type,cafe_restaurant_eating_out_cnt,unified_address,entertainment_cnt,...,site_id,healthcare_cnt,expect_demolition,longitude,ceiling_height,has_elevator,laundry_and_repair_services_cnt,build_year,city_quadkey,latlon
0,36.0,3031030817531317628,0,45.0,0,55.705219,BRICK,24.0,"Россия, Москва, 1-й Автозаводский проезд, 3",18.0,...,0,16.0,False,37.656994,0.0,True,38.0,1937,120310101302011,"(55.70521926879883, 37.6569938659668)"


In [7]:
preproc.dist_to_center(house.latlon)

,angle,dist
0,208.609050,5.662364
1,208.592932,5.708018
2,209.068190,5.747173
3,46.520739,8.484255
4,47.252273,8.448343
...,...,...
112198,229.115474,49.273623
112199,201.878450,75.988917
112200,187.834313,1.904622
112201,148.526656,8.873159


In [ ]:
preproc.closest_tube(house.latlon)